In [9]:
import pandas as pd
import numpy as np
import wfdb
import ast
from scipy.signal import find_peaks

In [10]:
nrows=1 # number of patients to load, set to None to load all patients
path="./"
sampling_rate=100

In [15]:
######################################
# Pan–Tompkins algorithm in python   #
# Author: Pramod kumar Chinthala     #
#                                    #
######################################

import numpy as np
from scipy.signal import butter, filtfilt


class Pan_tompkins:
    """ Implementationof Pan Tompkins Algorithm.

    Noise cancellation (bandpass filter) -> Derivative step -> Squaring and integration.

    Params:
        data (array) : ECG data
        sampling rate (int)
    returns:
        Integrated signal (array) : This signal can be used to detect peaks


    ----------------------------------------
    HOW TO USE ?
    Eg.

    ECG_data = [4, 7, 80, 78, 9], sampling  =2000
    
    call : 
       signal = Pan_tompkins(ECG_data, sampling).fit()

    ----------------------------------------
    
    """
    def __init__(self, data, sample_rate):

        self.data = data
        self.sample_rate = sample_rate


    def fit(self, normalized_cut_offs=None, butter_filter_order=2, padlen=150, window_size=None):
        ''' Fit the signal according to algorithm and returns integrated signal
        
        '''
        # 1.Noise cancellationusing bandpass filter
        self.filtered_BandPass = self.band_pass_filter(normalized_cut_offs, butter_filter_order, padlen)
        
        # 2.derivate filter to get slpor of the QRS
        self.derviate_pass = self.derivative_filter()

        # 3.Squaring to enhance dominant peaks in QRS
        self.square_pass = self.squaring()

        # 4.To get info about QRS complex
        self.integrated_signal = self.moving_window_integration( window_size)

        return self.integrated_signal


    def band_pass_filter(self, normalized_cut_offs=None, butter_filter_order=2, padlen=150):
        ''' Band pass filter for Pan tompkins algorithm
            with a bandpass setting of 5 to 20 Hz

            params:
                normalized_cut_offs (list) : bandpass setting canbe changed here
                bandpass filte rorder (int) : deffault 2
                padlen (int) : padding length for data , default = 150
                        scipy default value = 2 * max(len(a coeff, b coeff))

            return:
                filtered_BandPass (array)
        '''

        # Calculate nyquist sample rate and cutoffs
        nyquist_sample_rate = self.sample_rate / 2

        # calculate cutoffs
        if normalized_cut_offs is None:
            normalized_cut_offs = [5/nyquist_sample_rate, 15/nyquist_sample_rate]
        else:
            assert type(self.sample_rate ) is list, "Cutoffs should be a list with [low, high] values"

        # butter coefficinets 
        b_coeff, a_coeff = butter(butter_filter_order, normalized_cut_offs, btype='bandpass')[:2]

        # apply forward and backward filter
        filtered_BandPass = filtfilt(b_coeff, a_coeff, self.data, padlen=padlen)
        
        return filtered_BandPass


    def derivative_filter(self):
        ''' Derivative filter

        params:
            filtered_BandPass (array) : outputof bandpass filter
        return:
            derivative_pass (array)
        '''

        # apply differentiation
        derviate_pass= np.diff(self.band_pass_filter())

        return derviate_pass


    def squaring(self):
        ''' squaring application on derivate filter output data

        params:

        return:
            square_pass (array)
        '''

        # apply squaring
        square_pass= self.derivative_filter() **2

        return square_pass 


    def moving_window_integration(self, window_size=None):
        ''' Moving avergae filter 

        Params:
            window_size (int) : no. of samples to average, if not provided : 0.08 * sample rate
            sample_rate (int) : should be given if window_size is not given  
        return:
            integrated_signal (array)
        '''

        if window_size is None:
            assert self.sample_rate is not None, "if window size is None, sampling rate should be given"
            window_size = int(0.08 * int(self.sample_rate))  # given in paper 150ms as a window size
        

        # define integrated signal
        integrated_signal = np.zeros_like(self.squaring())

        # cumulative sum of signal
        cumulative_sum = self.squaring().cumsum()

        # estimationof area/ integral below the curve deifnes the data
        integrated_signal[window_size:] = (cumulative_sum[window_size:] - cumulative_sum[:-window_size]) / window_size

        integrated_signal[:window_size] = cumulative_sum[:window_size] / np.arange(1, window_size + 1)

        return integrated_signal

In [16]:
ECG_data = [4, 7, 80, 78, 9]
sampling  =2000

ValueError: The length of the input vector x must be greater than padlen, which is 150.

In [18]:
def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([signal for signal, meta in data])
    return data

# HRV calculation function (based on RR intervals)
def calculate_hrv(ecg_signal, sampling_rate):
    # Assuming the ECG signal is one-dimensional
    ecg = ecg_signal.flatten()

    # Find R-peaks using a simple peak detection (find local maxima)
    peaks, _ = find_peaks(ecg, distance=sampling_rate//2)  # Assume at least 0.5 sec between R peaks
    rr_intervals = np.diff(peaks) / sampling_rate * 1000  # RR intervals in milliseconds

    # Calculate HRV statistics
    if len(rr_intervals) > 1:
        hrv_mean = np.mean(rr_intervals)
        hrv_std = np.std(rr_intervals)
        hrv_min = np.min(rr_intervals)
        hrv_max = np.max(rr_intervals)
    else:
        # If only one RR interval, return NaN for std, min, max
        hrv_mean = np.mean(rr_intervals) if len(rr_intervals) > 0 else np.nan
        hrv_std = np.nan
        hrv_min = np.nan
        hrv_max = np.nan

    return hrv_mean, hrv_std, hrv_min, hrv_max

def add_hrv_features(df, X, sampling_rate):
    # Add columns for HRV mean, std, min, max to the dataframe
    hrv_means = []
    hrv_stds = []
    hrv_mins = []
    hrv_maxs = []

    for idx, signal in enumerate(X):
        hrv_mean, hrv_std, hrv_min, hrv_max = calculate_hrv(signal, sampling_rate)
        hrv_means.append(hrv_mean)
        hrv_stds.append(hrv_std)
        hrv_mins.append(hrv_min)
        hrv_maxs.append(hrv_max)

    df['hrv_mean'] = hrv_means
    df['hrv_std'] = hrv_stds
    df['hrv_min'] = hrv_mins
    df['hrv_max'] = hrv_maxs

    return df

# Load and convert annotation data
Y = pd.read_csv(path + 'ptbxl_database.csv', index_col='ecg_id', nrows=nrows)
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

# Load raw signal data
X = load_raw_data(Y, sampling_rate, path)

# Add HRV features to the DataFrame
Y = add_hrv_features(Y, X, sampling_rate)

database_row = Y
database_row.tail()


,patient_id,age,sex,height,weight,nurse,site,device,recording_date,report,...,electrodes_problems,extra_beats,pacemaker,strat_fold,filename_lr,filename_hr,hrv_mean,hrv_std,hrv_min,hrv_max
ecg_id,,,,,,,,,,,,,,,,,,,,,
1,15709.0,56.0,1,NaN,63.0,2.0,0.0,CS-12 E,1984-11-09 09:17:34,sinusrhythmus periphere niederspannung,...,NaN,NaN,NaN,3,records100/00000/00001_lr,records500/00000/00001_hr,651.195652,128.961929,500.0,1000.0


In [19]:
X_train[0][0]

array([-0.119, -0.055,  0.064,  0.086, -0.091,  0.004, -0.069, -0.031,
        0.   , -0.026, -0.039, -0.079])

In [20]:
Pan_tompkins(X_train[0][0], sampling_rate).fit()

ValueError: The length of the input vector x must be greater than padlen, which is 150.